# Use network isolation with managed online endpoints

The following prerequisites are needed for this example: 

Install them using the following code:

In [ ]:
%pip install azure-mgmt-resource
%pip install azure-mgmt-compute
%pip install azure-mgmt-containerregistry

## Connect to Azure Machine Learning

### Import Required Libraries

In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.mgmt.resource import ResourceManagementClient, DeploymentScriptsClient
from azure.mgmt.resource.templatespecs import TemplateSpecsClient
from azure.mgmt.resource.templatespecs.models import TemplateSpec
from azure.mgmt.resource.resources.v2021_04_01 import models as resource_models
from azure.mgmt.resource.templatespecs.v2022_02_01.models import TemplateSpec
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.compute.models import VirtualMachineRunCommand, VirtualMachine, VirtualMachineRunCommandScriptSource,RunCommandInputParameter
from azure.identity import DefaultAzureCredential, ManagedIdentityCredential
import os, json, random
from pathlib import Path

### Set variables

In [2]:
subscription = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"
location = "<LOCATION>"

image_name="img"

_rand = random.randint(0,10000)
suffix = f"vnet{_rand}"
acr_name=f"cr{suffix}"
workspace=f"mlw-{suffix}"
identity_name=f"uai{suffix}"

vm_name="moevnet-vm"
vnet_name=f"vnet-{suffix}"
subnet_name="snet-scoring"

endpoint_name=f"endpt-vnet-{_rand}"

## Get clients

In [5]:
credential = DefaultAzureCredential()

ml_client = MLClient(credential, subscription, resource_group, workspace)

resource_client = ResourceManagementClient(credential, subscription)

compute_client = ComputeManagementClient(credential, subscription)

ts_client = TemplateSpecsClient(credential, subscription)

## Create the workspace and secured resources

### Create identity

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{suffix}-uai",
    parameters={"properties" : {
        "mode" : "Incremental",
        "template" : json.loads(Path("vnet/setup_ws/uai.json").read_text()), 
        "parameters" : {
            "suffix" : {
                "value" : f"{suffix}"},
        }
        }
    }
).result()

### Create workspace

In [ ]:
tempspec_root = Path("vnet/setup_ws")

tempspec = ts_client.template_spec_versions.create_or_update(
    template_spec_name="test",
    template_spec_version=str(_rand),
    resource_group_name=resource_group,
    template_spec_version_model={
        "location": "eastus",
        "linked_templates": [
            {
                "template": json.loads(f.read_text()),
                "path": str(f.relative_to(tempspec_root)),
            }
            for f in (tempspec_root / "modules").iterdir()
        ],
        "main_template": json.loads((tempspec_root / "main-linked.json").read_text()),
    }
).result()

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=suffix,
    parameters={ "properties" :{
            "mode" : "Incremental",
            "template_link": {
                "id" : tempspec.id
            },
            "parameters" : {
                "suffix": {
                    "value" : suffix
                }
            }
        }
    }
).result()

## Create the Virtual Machine Jump Box

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{vm_name}-{endpoint_name}",
    parameters={"properties" : {
        "mode" : "Incremental",
        "template" : json.loads(Path("vnet/setup_vm/vm-main.json").read_text()), 
        "parameters" : {
            "vmName" : { "value" : vm_name },
            "vnetName" : {"value" : vnet_name},
            "identityName" : {"value" : identity_name},
            "subnetName" : {"value" : subnet_name},
            }
        }
    }
).result()

## Configure the VM - Using VM Commands
The following scripts use the `RunShellScript` VM command to run Python scripts that setup the VM, create a Managed Online Endpoint, and score it. The scripts can also be run using ssh using the commands in `ssh_commands.sh` in the section below.

### Execute the setup script

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "source": {"script": Path("vnet/setup_vm/scripts/vmsetup.sh").read_text()},
        "command_type": "RunShellScript",
        "parameters": [
            {"name" : "SUBSCRIPTION",
            "value" : subscription},
            {"name" : "RESOURCE_GROUP",
            "value" : resource_group},
            {"name" : "LOCATION",
            "value" : location},
            {"name" : "WORKSPACE",
            "value" : workspace},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name}
        ]
    }
).result()

### Build Image

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "source": {"script": Path("vnet/setup_vm/scripts/run_py.sh").read_text()},
        "command_type": "RunShellScript",
        "parameters": [
            {"name" : "SCRIPT_NAME",
            "value" : "build_image.py"},
            {"name" : "SUBSCRIPTION",
            "value" : subscription},
            {"name" : "RESOURCE_GROUP",
            "value" : resource_group},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name},
            {"name" : "LOCATION",
            "value" : location},
            {"name" : "WORKSPACE",
            "value" : workspace},
            {"name" : "ACR_NAME",
            "value" : acr_name},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name}
        ]
    }
).result()

### Create the Managed Online Endpoint

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "source": {"script": Path("vnet/setup_vm/scripts/run_py.sh").read_text()},
        "command_type": "RunShellScript",
        "parameters": [
            {"name" : "SCRIPT_NAME",
            "value" : "create_moe.py"},
            {"name" : "SUBSCRIPTION",
            "value" : subscription},
            {"name" : "RESOURCE_GROUP",
            "value" : resource_group},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name},
            {"name" : "LOCATION",
            "value" : location},
            {"name" : "WORKSPACE",
            "value" : workspace},
            {"name" : "ACR_NAME",
            "value" : acr_name},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name}
        ]
    }
).result()

## Configure the VM - Interactively Over SSH 

### Connect via SSH

In [ ]:
ssh $USER@$HOST -t "export SUBSCRIPTION=<SUBSCRIPTION>; export RESOURCE_GROUP=<RESOURCE_GROUP>; bash -l" 

### Setup VM

In [ ]:
export USER=$(whoami)
sudo apt-get update -y && sudo apt install wget -y
sudo mkdir -p /home/$USER/samples; sudo git clone -b $GIT_BRANCH --depth 1 https://github.com/Azure/azureml-examples.git /home/$USER/samples/azureml-examples -q

cd /home/$USER/samples/azureml-examples/sdk/python/endpoints/online/managed/vnet/setup_vm/scripts

wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh 
sudo chmod +x Miniconda3-latest-Linux-x86_64.sh
sudo ./Miniconda3-latest-Linux-x86_64.sh -b -p /opt/anaconda
eval "$(/opt/anaconda/bin/conda shell.bash hook)"
conda create -n vnet python=3.10.4 -y
conda activate vnet

pip install azure-ai-ml azure-mgmt-containerregistry azure-storage-blob

eval "$(/opt/anaconda/bin/conda shell.bash hook)"
conda activate vnet

### Build Image

In [ ]:
python build_image.py

### Create a secured managed online endpoint

In [ ]:
python create_moe.py

## Cleanup

### Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name).wait()

### Delete the VM

In [ ]:
compute_client.virtual_machines.begin_delete(resource_group_name=resource_group, vm_name=vm_name).wait()